In [1]:
import csv
import os
import scipy.io
import cPickle as pickle
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import itertools
import numpy as np
import pandas as pd
from collections import OrderedDict
import shutil

In [2]:
data_folder = 'mm10'
matrix_file = 'matrix.mtx'
genes_file = 'genes.tsv'
barcodes_file = 'barcodes.tsv'

matrix_file_path = os.path.join(data_folder,matrix_file)
genes_file_path = os.path.join(data_folder,genes_file)
barcodes_file_path = os.path.join(data_folder,barcodes_file)


In [3]:
# load matrix file
# takes time
mat = scipy.io.mmread(matrix_file_path)

In [4]:
mat_dense = mat.todense()

In [5]:
#load genes
genes_path = os.path.join(data_folder, "genes.tsv")
gene_ids = [row[0] for row in csv.reader(open(genes_path), delimiter="\t")]
gene_names = [row[1] for row in csv.reader(open(genes_path), delimiter="\t")]

In [6]:
#load barcodes
barcodes_path = os.path.join(data_folder, "barcodes.tsv")
barcodes = [row[0] for row in csv.reader(open(barcodes_path), delimiter="\t")]

In [7]:
print len(barcodes)
# barcodes => columns
# gene => rows

23367


In [8]:
mat_df = pd.DataFrame(mat_dense)

In [9]:
print len(mat_df.index)

27998


In [10]:
#index in row
Il10_ind = gene_names.index('Il10')
Il10ra_ind = gene_names.index('Il10ra')
print Il10_ind
print Il10ra_ind

972
18118


# Il10


In [11]:
#create Il10 df
df = pd.DataFrame(mat_dense)

In [12]:
df.shape

(27998, 23367)

In [20]:
df.loc[Il10ra_ind][Il10ra_ind]

0

In [75]:
df.max(axis=0)

0         462
1         924
2         111
3         234
4          84
5        1283
6         464
7         136
8         106
9         416
10        183
11        174
12         96
13        255
14        171
15        141
16        107
17         63
18         48
19        107
20        282
21        206
22        181
23         90
24        284
25        294
26        207
27        112
28         86
29        134
         ... 
23337      75
23338      98
23339     291
23340      87
23341      77
23342     149
23343      95
23344      85
23345     256
23346     271
23347     222
23348     402
23349     121
23350     448
23351      97
23352     138
23353     146
23354      79
23355    1335
23356     316
23357     155
23358     111
23359     169
23360     122
23361     361
23362     269
23363     103
23364     130
23365     125
23366     168
Length: 23367, dtype: int64

In [91]:

df[df.columns] *= 10000

MemoryError: 

In [21]:
#normalize df
from sklearn import preprocessing

x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_norm = pd.DataFrame(x_scaled)
mat_df_Il10 = df_norm

C:\Users\Rong Lu\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [22]:
Il10_row = mat_df_Il10.loc[Il10_ind]
Il10ra_row = mat_df_Il10.loc[Il10ra_ind]

In [18]:
type(Il10_row)

pandas.core.series.Series

In [59]:
Il10_df = Il10_row.to_frame().T
Il10ra_df = Il10ra_row.to_frame().T

In [60]:
comb_df = Il10_df.append(Il10ra_df)

In [61]:
comb_df

,0,1,2,3,4,5,6,7,8,9,...,23357,23358,23359,23360,23361,23362,23363,23364,23365,23366
972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
comb_df.max(axis = 1)

972      0.016000
18118    0.028571
dtype: float64

In [63]:
comb_df.index = ['Il10','Il10ra']
comb_df

,0,1,2,3,4,5,6,7,8,9,...,23357,23358,23359,23360,23361,23362,23363,23364,23365,23366
Il10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Il10ra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
comb_df[comb_df.columns] +=1

In [88]:
comb_df_log = comb_df.apply(np.log10, axis = 0)

In [89]:
comb_df_log.max(axis=1)

Il10      0.006894
Il10ra    0.012234
dtype: float64

In [69]:
writer = pd.ExcelWriter('Il10 and Il10ra rows norm + log.xlsx')
comb_df_log.to_excel(writer)
writer.save()